In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sujaykapadnis/tomato-maturity-detection-and-quality-grading")

print("Path to dataset files:", path)

C:\Program Files\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|█████████████████████████████████████████████████████████████████████████████| 5.08G/5.08G [11:15<00:00, 8.08MB/s]

Extracting files...


Path to dataset files: C:\Users\vasqu\.cache\kagglehub\datasets\sujaykapadnis\tomato-maturity-detection-and-quality-grading\versions\1


In [2]:
print("Path to dataset files:", path)

Path to dataset files: C:\Users\vasqu\.cache\kagglehub\datasets\sujaykapadnis\tomato-maturity-detection-and-quality-grading\versions\1


In [44]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# --- Configuración ---
IMAGE_SIZE = (128, 128)
BATCH_SIZE = 32
EPOCHS = 3
BASE_DIR   = r"C:\Users\vasqu\.cache\kagglehub\datasets\sujaykapadnis\tomato-maturity-detection-and-quality-grading\versions\1\Tomato Maturity Detection and Quality Grading Dataset"

# Rutas a los dos sub‑datasets
MATURITY_DIR = os.path.join(BASE_DIR, "Tomato Maturity Detection Dataset","Tomato Maturity Detection Dataset", "Original Dataset")
QUALITY_DIR  = os.path.join(BASE_DIR, "Tomato Quality Grading Dataset", "Tomato Quality Grading Dataset", "Original Dataset")

# Creamos un generador con split para train/val
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)

# 1) Generators para MADUREZ (Immature vs Mature)
train_mat = datagen.flow_from_directory(
    MATURITY_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    classes=['Immature','Mature'],
    class_mode='binary',
    subset='training'
)
val_mat = datagen.flow_from_directory(
    MATURITY_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    classes=['Immature','Mature'],
    class_mode='binary',
    subset='validation'
)

# 2) Generators para CALIDAD (Rotten vs Fresh)
train_qual = datagen.flow_from_directory(
    QUALITY_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    classes=['Rotten','Fresh'],
    class_mode='binary',
    subset='training'
)
val_qual = datagen.flow_from_directory(
    QUALITY_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    classes=['Rotten','Fresh'],
    class_mode='binary',
    subset='validation'
)

# Función para crear la misma arquitectura
def build_binary_model(input_shape=(*IMAGE_SIZE, 3)):
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        MaxPooling2D(2,2),
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Conv2D(128, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# — Entrenar modelo de MADUREZ —
model_mat = build_binary_model()
model_mat.fit(
    train_mat,
    epochs=EPOCHS,
    validation_data=val_mat
)
model_mat.save('tomato_maturity_model.h5')


# — Entrenar modelo de CALIDAD —
model_qual = build_binary_model()
model_qual.fit(
    train_qual,
    epochs=EPOCHS,
    validation_data=val_qual
)
model_qual.save('tomato_quality_model.h5')

Found 4000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 6389 images belonging to 2 classes.
Found 1597 images belonging to 2 classes.
Epoch 1/3
125/125 ━━━━━━━━━━━━━━━━━━━━ 559s 4s/step - accuracy: 0.7795 - loss: 0.4424 - val_accuracy: 0.8360 - val_loss: 0.5206
Epoch 2/3
125/125 ━━━━━━━━━━━━━━━━━━━━ 558s 4s/step - accuracy: 0.9700 - loss: 0.1145 - val_accuracy: 0.9700 - val_loss: 0.0765
Epoch 3/3
125/125 ━━━━━━━━━━━━━━━━━━━━ 520s 4s/step - accuracy: 0.9722 - loss: 0.0786 - val_accuracy: 0.9830 - val_loss: 0.0530


Epoch 1/3
200/200 ━━━━━━━━━━━━━━━━━━━━ 899s 4s/step - accuracy: 0.5722 - loss: 0.6983 - val_accuracy: 0.7683 - val_loss: 0.4632
Epoch 2/3
200/200 ━━━━━━━━━━━━━━━━━━━━ 917s 5s/step - accuracy: 0.7524 - loss: 0.4949 - val_accuracy: 0.8892 - val_loss: 0.3014
Epoch 3/3
200/200 ━━━━━━━━━━━━━━━━━━━━ 916s 5s/step - accuracy: 0.8775 - loss: 0.3159 - val_accuracy: 0.9436 - val_loss: 0.1846


In [45]:
print("hola")

hola


In [5]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model

# --- Configuración ---
IMAGE_SIZE = (128, 128)
THRESHOLD = 0.5

# Ajusta aquí la ruta absoluta a tu escritorio y al nombre exacto de tu archivo:
IMG_PATH = r"C:\Users\urbin\PROYECTO IA\prueba1.jpg"   # o .png, .jpeg, etc
# Rutas a los modelos que ya entrenaste y guardaste
MATURITY_MODEL_PATH = 'tomato_maturity_model.h5'
QUALITY_MODEL_PATH  = 'tomato_quality_model.h5'

# Mapas de etiquetas
MATURITY_LABELS = {0: 'Immature', 1: 'Mature'}
QUALITY_LABELS  = {0: 'Rotten',   1: 'Fresh'}

# --- Funciones de ayuda ---
def preprocess_image(img_path, target_size=IMAGE_SIZE):
    img = load_img(img_path, target_size=target_size)
    x   = img_to_array(img) / 255.0
    return np.expand_dims(x, axis=0)

def predict_tomato(img_path, maturity_model, quality_model):
    x = preprocess_image(img_path)
    m_pred = maturity_model.predict(x)[0][0]
    q_pred = quality_model.predict(x)[0][0]
    m_cls  = 1 if m_pred > THRESHOLD else 0
    q_cls  = 1 if q_pred > THRESHOLD else 0
    return MATURITY_LABELS[m_cls], QUALITY_LABELS[q_cls]

# --- Carga modelos ---
if not os.path.exists(MATURITY_MODEL_PATH) or not os.path.exists(QUALITY_MODEL_PATH):
    raise FileNotFoundError("Asegúrate de tener ambos archivos .h5 en el mismo directorio que este script.")

maturity_model = load_model(MATURITY_MODEL_PATH)
quality_model  = load_model(QUALITY_MODEL_PATH)

# --- Predicción directa ---
maturity, quality = predict_tomato(IMG_PATH, maturity_model, quality_model)
print(f'🧐 Resultado para "{IMG_PATH}":')
print(f'  • Madurez: {maturity}')
print(f'  • Calidad: {quality}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 259ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
🧐 Resultado para "C:\Users\urbin\PROYECTO IA\prueba1.jpg":
  • Madurez: Mature
  • Calidad: Rotten
